# Important Implementation Note
I've defined 2 separate GAN's below: a basic GAN and a DCGAN. The following code is compatible with both networks. If you wish to switch between the different implementations, I've indicated whether to comment or uncomment code at the necessary locations. 

# Defining The Accelerator
Using a GPU if available, else a CPU. In this case, satisfactory performance is obtained with a CPU and training time is not too high.

In [ ]:
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import DataLoader, TensorDataset

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#Loading the training data
train = pd.read_csv('../input/mnist-in-csv/mnist_train.csv')

# Defining the Training Data
From the original MNIST dataset, 60000 images are chosen for training. The training data is reshaped into 28x28 numpy array to represent the grayscale images. Finally, an important step of normalization follows to allow the GAN to converge better. 

In [ ]:
train_data = train.drop(labels = ['label'], axis = 1)
train_data = train_data.values.reshape(-1, 28, 28)
train_data = train_data/255.0

#To create some space
del train 

In [ ]:
#Just checking if the data type is as expected
print(isinstance(train_data, np.ndarray))

#Checking the sanity of the shape of the training data
print(train_data.shape)

# Visualizing the Data
Randomly plotting one of the training examples to catch a drift about the training data.

In [ ]:
plt.imshow(train_data[7], cmap = 'gray')

# Using DataLoader
To allow for efficient iteration over the different mini-batches using a hard coded batch size of 32.

In [ ]:
#Converting to torch tensor
train_data = torch.Tensor(train_data)

#Checking to confirm dtype as tensor
print(isinstance(train_data, torch.Tensor))

random_seed = 1
batch_size = 32
train_dl = DataLoader(train_data, batch_size, shuffle = True)

# Noise Generation
The input to the generator in a GAN is usually random noise. Here, I use torch.rand to generate random numbers between 0 and 1 and then rescale the numbers to fall between -1 and 1 so as to make the mean 0 for the uniform distribution to facilitate the training of the GAN.

In [ ]:
def random_noise_generator(batch_size, dim):
    return torch.rand(batch_size, dim)*2 - 1

In [ ]:
#Just checking the noise generator and plotting one of its outputs
a = random_noise_generator(64, 100)
b = a[2]
b = b.reshape(10, 10)
b = b.numpy()
plt.imshow(b, cmap = 'gray')

# Discriminator Model for Basic GAN
The discriminator model is fairly straightforward having a few fully connected layers having one small caveat, instead of the usual ReLU for activation, here, I've used a **leaky ReLU**. This is because, a leaky ReLU has a small gradient for negative values of input, allowing for a stronger flow of gradients from the discriminator to the generator. Also, some dropout layers have been added in intermediate stages to prevent overfitting. 
Further, the final output is supposed to be between 0 and 1 (to predict if real or fake), however, the loss function used later on is **BCEWithLogitsLoss** (for numerical stability) which combines the sigmoid with the normal binary cross entropy function, so, no activation function is applied to the output.

In [ ]:
#Unomment this if you want to run the Basic GAN instead of the DCGAN

"""
class Discriminator(nn.Module):
    
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.fc1 = nn.Linear(input_size, hidden_size*4)
        self.fc2 = nn.Linear(hidden_size*4, hidden_size*2)
        self.fc3 = nn.Linear(hidden_size*2, hidden_size)
        self.fc4 = nn.Linear(hidden_size, output_size)
        
        self.dropout = nn.Dropout(0.3)
        
    def forward(self, x):
        x = x.view(-1, 28*28)
        x = F.leaky_relu(self.fc1(x), 0.2)
        x = self.dropout(x)
        x = F.leaky_relu(self.fc2(x), 0.2)
        x = self.dropout(x)
        x = F.leaky_relu(self.fc3(x), 0.2)
        x = self.dropout(x)
        
        out = self.fc4(x)
        return out
"""        

# Discriminator Model for DCGAN
The discriminator model for a DCGAN can be implemented in many different ways, but it's recommended by the authors of the original paper on DCGAN's to have **strided convolutions** for downscaling, remove any fully connected layers, use batch normalization and the **leaky ReLU** for activation of all layers except sigmoid for the last one. I've added a few dropout layers additionally to prevent overfitting.

In [ ]:
#Comment this if you want to run the Basic GAN instead of the DCGAN

class Discriminator(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.conv0 = nn.Conv2d(1, 32, kernel_size = 3, stride = 2, padding = 1)
        #self.conv0_bn = nn.BatchNorm2d(32)
        self.conv0_drop = nn.Dropout2d(0.25)
        self.conv1 = nn.Conv2d(32, 64, kernel_size = 3, stride = 1, padding = 1)
        #self.conv1_bn = nn.BatchNorm2d(64)
        self.conv1_drop = nn.Dropout2d(0.25)
        self.conv2 = nn.Conv2d(64, 128, kernel_size = 3, stride = 1, padding = 1)
        #self.conv2_bn = nn.BatchNorm2d(128)
        self.conv2_drop = nn.Dropout2d(0.25)
        self.conv3 = nn.Conv2d(128, 256, kernel_size = 3, stride = 2, padding = 1)
        #self.conv3_bn = nn.BatchNorm2d(256)
        self.conv3_drop = nn.Dropout2d(0.25)
        self.fc = nn.Linear(12544, 1)
    
    def forward(self, x):
        x = x.view(-1, 1, 28, 28)
        x = F.leaky_relu(self.conv0(x), 0.2)
        #x = self.conv0_bn(x)
        x = self.conv0_drop(x)
        x = F.leaky_relu(self.conv1(x), 0.2)
        #x = self.conv1_bn(x)
        x = self.conv1_drop(x)
        x = F.leaky_relu(self.conv2(x), 0.2)
        #x = self.conv2_bn(x)
        x = self.conv2_drop(x)
        x = F.leaky_relu(self.conv3(x), 0.2)
        #x = self.conv3_bn(x)
        x = self.conv3_drop(x)
        x = x.view(-1, self.num_flat_features(x))
        x = self.fc(x)
        
        return x
    
    def num_flat_features(self, x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        
        return num_features

# Generator Model for Basic GAN
The generator model is quite similar to the discriminator, with two small differences:
1. Instead of the input being downscaled, it's upscaled to reach expected dimensions of MNIST dataset images.
2. Final activation function **tanh** is applied to the output bringing its values between -1 and 1 for best performance.

In [ ]:
#Uncomment this if you want to run the Basic GAN instead of the DCGAN

"""
class Generator(nn.Module):
    
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size*2)
        self.fc3 = nn.Linear(hidden_size*2, hidden_size*4)
        self.fc4 = nn.Linear(hidden_size*4, output_size)
        
        self.dropout = nn.Dropout(0.3)
        
    def forward(self, x):
        x = F.leaky_relu(self.fc1(x), 0.2)
        x = self.dropout(x)
        x = F.leaky_relu(self.fc2(x), 0.2)
        x = self.dropout(x)
        x = F.leaky_relu(self.fc3(x), 0.2)
        x = self.dropout(x)
        
        out = torch.tanh(self.fc4(x))
        return out
"""        

# Generator Model for DCGAN
Similar to the discrminator, there are many possible implementations but the authors of the original paper on DCGAN's recommend using **transposed convolutions** for upscaling, using batch normalization in intermediate stages and having a **ReLU** activation function after all layers except a tanh activation for the output.

In [ ]:
#Comment this if you want to run the Basic GAN instead of the DCGAN

class Generator(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.fc = nn.Linear(100, 256*7*7)
        self.trans_conv1 = nn.ConvTranspose2d(256, 128, kernel_size = 3, stride = 2, padding = 1, output_padding = 1)
        #self.trans_conv1_bn = nn.BatchNorm2d(128)
        self.trans_conv2 = nn.ConvTranspose2d(128, 64, kernel_size = 3, stride = 1, padding = 1)
        #self.trans_conv2_bn = nn.BatchNorm2d(64)
        self.trans_conv3 = nn.ConvTranspose2d(64, 32, kernel_size = 3, stride = 1, padding = 1)
        #self.trans_conv3_bn = nn.BatchNorm2d(32)
        self.trans_conv4 = nn.ConvTranspose2d(32, 1, kernel_size = 3, stride = 2, padding = 1, output_padding = 1)
    
    def forward(self, x):
        x = self.fc(x)
        x = x.view(-1, 256, 7, 7)
        x = F.relu(self.trans_conv1(x))
        #x = self.trans_conv1_bn(x)
        x = F.relu(self.trans_conv2(x))
        #x = self.trans_conv2_bn(x)
        x = F.relu(self.trans_conv3(x))
        #x = self.trans_conv3_bn(x)
        x = self.trans_conv4(x)
        x = torch.tanh(x)
        
        return x        

# Setting Model Hyperparameters for Basic GAN
Input size to the generator is hard coded to be 100. Can be manipulated to check variations in performance.

In [ ]:
#Uncomment this if you want to run the Basic GAN instead of the DCGAN

"""
#For Discriminator
input_disc = 784
hidden_disc = 32
output_disc = 1

#For Generator
input_gen = 100
hidden_gen = 32
output_gen = 784

"""

In [ ]:
#Uncomment this if you want to run the Basic GAN instead of the DCGAN
"""
#Creating instances of models
D = Discriminator(input_disc, hidden_disc, output_disc)
G = Generator(input_gen, hidden_gen, output_gen)
"""

#Comment the following 2 lines if you want to run the Basic GAN instead of the DCGAN
D = Discriminator()
G = Generator()

#Sanity check of the model instances
print(D)
print(G)

#Passing to the GPU
D = D.to(device)
G = G.to(device)

D = D.float()
G = G.float()

# Discriminator Loss
Discriminator loss is the resultant sum of two components:
1. On passing real training set images through the discriminator, the output values are compared with 1 (for real images) and real loss is calculated.
2. On passing random noise through the generator, fake images are created which are further passed through the discriminator. These outputs are compared with 0 (for fake images) and fake loss is calculated.

I have defined relevant functions to calculate real loss, fake loss and the resultant sum of these two losses.

In [ ]:
Loss = nn.BCEWithLogitsLoss()
def discriminator_real_loss(real_out):
    real_label = torch.ones(real_out.size()[0], 1).to(device)
    real_loss = Loss(real_out.squeeze(), real_label.squeeze())
    return real_loss

def discriminator_fake_loss(fake_out):
    fake_label = torch.zeros(fake_out.size()[0], 1).to(device)
    fake_loss = Loss(fake_out.squeeze(), fake_label.squeeze())
    return fake_loss

def discriminator_loss(real_out, fake_out):
    real_loss = discriminator_real_loss(real_out)
    fake_loss = discriminator_fake_loss(fake_out)
    total_loss = (real_loss + fake_loss)
    return total_loss

# Generator Loss
Passing random noise through the generator, we get fake images. On further passing these through the discriminator, we get their output labels. Since we want the generator to fool the discriminator, these output labels are compared with 1 (for supposedly real images) and loss is calculated.

In [ ]:
def generator_loss(gen_disc_out):
    label = torch.ones(gen_disc_out.size()[0], 1).to(device)
    gen_loss = Loss(gen_disc_out.squeeze(), label.squeeze())
    return gen_loss

# Defining Optimizers
If going with the basic GAN:
Using Adam optimizers for both discriminator and generator. The learning rate is adjusted to 0.002, a slight deviation from the original paper, to allow for faster learning in this case.

If using the DCGAN architecture:
Using Adam optimizers for the generator and discriminator with a slight deviation from the usual settings, setting the learning rate to 0.0002 and $\beta_{1}$ to be 0.5 according to the authors of the DCGAN paper for best performance.

In [ ]:
#Important: If using Basic GAN instead of DCGAN, go for standard values lr = 0.001 and betas = (0.9, 0.999) 

disc_opt = optim.Adam(D.parameters(), lr = 0.0002, betas = (0.5, 0.999))
gen_opt = optim.Adam(G.parameters(), lr = 0.0002, betas = (0.5, 0.999))

# Training the GAN
The GAN was trained with a mini-batch size of 32 for 25 epochs and samples of the generator output were printed every 5 epochs to monitor the progress of the generator.

In [ ]:
def train(D, G, disc_opt, gen_opt, train_dl, batch_size = 32, epochs = 25, gen_input_size = 100):
    
    disc_losses = []
    gen_losses = []
    
    #Having a fixed sample to monitor the progress of the generator
    sample_size = 16
    fixed_samples = random_noise_generator(sample_size, gen_input_size)
    fixed_samples = fixed_samples.to(device)
    
    #Going into training mode
    D.train()
    G.train()
    
    for epoch in range(epochs + 1):
        
        disc_loss_total = 0
        gen_loss_total = 0
        gen_out = 0
        
        for train_x in train_dl:
            
            #Discriminator training
            disc_opt.zero_grad()
            
            train_x = train_x*2 - 1          #Converting the real images to have values between -1 and 1
            train_x = train_x.to(device)     #Passing to GPU
            real_out = D(train_x.float())    
            
            disc_gen_in = random_noise_generator(batch_size, gen_input_size)
            disc_gen_in = disc_gen_in.to(device)   #Passing to GPU
            
            disc_gen_out = G(disc_gen_in.float()).detach()  #Detaching to avoid training the generator
            fake_out = D(disc_gen_out.float())
            
            disc_loss = discriminator_loss(real_out, fake_out)  #Loss calculation
            disc_loss_total += disc_loss
            disc_loss.backward()
            disc_opt.step()  
        
            #Generator training
            gen_opt.zero_grad()
            
            
            gen_out = G(disc_gen_in.float())     #Feeding noise into the generator
            gen_disc_out = D(gen_out.float())       #Passing into the discrminator
            
            gen_loss = generator_loss(gen_disc_out)  #Generator loss calculation
            gen_loss_total += gen_loss
            gen_loss.backward()
            gen_opt.step()
        
        disc_losses.append(disc_loss_total)
        gen_losses.append(gen_loss_total)
        
        #Plotting samples every 5 epochs
        if epoch%5 == 0:
            G.eval()                    #Going into eval mode to get sample images         
            samples = G(fixed_samples.float())
            G.train()                   #Going back into train mode
            
            fig, axes = plt.subplots(figsize=(7,7), nrows=4, ncols=4, sharey=True, sharex=True)
            for ax, img in zip(axes.flatten(), samples):
               img = img.cpu().detach()
               ax.xaxis.set_visible(False)
               ax.yaxis.set_visible(False)
               im = ax.imshow(img.reshape((28,28)), cmap='Greys_r')
        
        
        #Printing losses every epoch
        print("Epoch ", epoch, ": Discriminator Loss = ", disc_loss_total/len(train_dl), ", Generator Loss = ", gen_loss_total/len(train_dl))    
    
    return disc_losses, gen_losses

In [ ]:
disc_losses, gen_losses = train(D, G, disc_opt, gen_opt, train_dl, batch_size)

# Plotting Losses
The discriminator and generator losses are plotted as a function of the number of epochs to observe the trend and allow for easier debugging.

In [ ]:
fig, ax = plt.subplots()
disc_losses = np.array(disc_losses)
gen_losses = np.array(gen_losses)
plt.plot(disc_losses, label='Discriminator')
plt.plot(gen_losses, label='Generator')
plt.title("Training Losses")
plt.legend()